In [26]:
import pandas as pd

# added to suppress warnings coming from pyLDAvis
import warnings
warnings.filterwarnings('ignore')

In [27]:
data = pd.read_csv('data.csv', sep=';')
data.head()

,Categoria,Palabras clave,Frecuencia al interior del grupo,Frecuencia entre los grupos
0,Operativo,prodiccion,2,1
1,Operativo,parrilleros,1,1
2,Operativo,ingmecanico,1,1
3,Operativo,documentacion,6,1
4,Servicio al cliente,weekends,2,2


In [28]:
data[data['Frecuencia entre los grupos']<=11].sort_values(by=['Categoria', 'Frecuencia al interior del grupo'], ascending=[True, False])

,Categoria,Palabras clave,Frecuencia al interior del grupo,Frecuencia entre los grupos
10363,Agropecuario,agronomo,50,3
9410,Agropecuario,veterinario,30,4
7085,Agropecuario,agropecuario,29,3
2902,Agropecuario,credito,13,9
2009,Agropecuario,veterinaria,7,4
...,...,...,...,...
11932,Ventas - Comercial - Administración,informatica,1,5
11945,Ventas - Comercial - Administración,promovendedor,1,1
11969,Ventas - Comercial - Administración,contenido,1,8
12010,Ventas - Comercial - Administración,estatales,1,1


In [29]:
#Se filtran las categorías con menos de n elementos
n=17
category_filter = data.groupby('Categoria')['Palabras clave'].count().to_frame().sort_values(by='Palabras clave', ascending = False)
category_filter = category_filter[category_filter['Palabras clave']<n]
filtered_data = data[data['Categoria'].isin(category_filter.reset_index()['Categoria'].tolist())==False]
filtered_data

,Categoria,Palabras clave,Frecuencia al interior del grupo,Frecuencia entre los grupos
0,Operativo,prodiccion,2,1
1,Operativo,parrilleros,1,1
2,Operativo,ingmecanico,1,1
3,Operativo,documentacion,6,1
4,Servicio al cliente,weekends,2,2
...,...,...,...,...
12039,Operativo,tebaida,1,1
12040,Talento TI,jquery,1,1
12041,Operativo,pais,1,2
12042,Talento TI,pais,4,2


In [ ]:
data[data['Palabras clave'].map(len)<=3]['Palabras clave'].unique()

In [45]:
set(filtered_data['Categoria'].unique())

{'Agropecuario',
 'Compras',
 'Construcción',
 'Contaduria',
 'Diseño',
 'Educación',
 'Financiero',
 'Gastronomia',
 'Gestión de Calidad',
 'Juridicos',
 'Logistica',
 'Marketing',
 'Operativo',
 'Practicantes - Aprendices',
 'Project Manager',
 'Psicologia',
 'Recursos Humanos',
 'SST',
 'Salud',
 'Servicio al cliente',
 'Talento TI',
 'Ventas - Comercial - Administración'}

In [43]:
resulting_df = pd.DataFrame(columns=filtered_data.columns)
for category in filtered_data['Categoria'].unique():
    df_new = filtered_data[filtered_data['Categoria']==category]
    df_new['inter_rank'] = df_new['Frecuencia al interior del grupo'].rank(pct = True)
    df_new['exter_rank'] = df_new['Frecuencia entre los grupos'].rank(pct = True)
    resulting_df = pd.concat([resulting_df, df_new])

resulting_df['total_rank'] = resulting_df.apply(lambda row: row.inter_rank * (1-row.exter_rank), axis=1)
resulting_df

,Categoria,Palabras clave,Frecuencia al interior del grupo,Frecuencia entre los grupos,inter_rank,exter_rank,total_rank
0,Operativo,prodiccion,2,1,0.473763,0.271064,0.345343
1,Operativo,parrilleros,1,1,0.203898,0.271064,0.148629
2,Operativo,ingmecanico,1,1,0.203898,0.271064,0.148629
3,Operativo,documentacion,6,1,0.715742,0.271064,0.521730
7,Operativo,llamados,1,1,0.203898,0.271064,0.148629
...,...,...,...,...,...,...,...
11287,Juridicos,gestor,1,17,0.295455,0.810606,0.055957
11425,Juridicos,ingenieros,1,7,0.295455,0.560606,0.129821
11662,Juridicos,financiero,4,10,0.825758,0.636364,0.300275
11785,Juridicos,administrativo,2,16,0.674242,0.780303,0.148129


In [47]:
agro = resulting_df[resulting_df.Categoria == 'Agropecuario']
agro = agro.sort_values(["total_rank", "inter_rank"], ascending=False)

In [48]:
agro.head(10)

,Categoria,Palabras clave,Frecuencia al interior del grupo,Frecuencia entre los grupos,inter_rank,exter_rank,total_rank
661,Agropecuario,tradicionalcupo,3,1,0.761062,0.075221,0.703814
1309,Agropecuario,agronoma,3,1,0.761062,0.075221,0.703814
3601,Agropecuario,engorde,3,1,0.761062,0.075221,0.703814
4494,Agropecuario,granjas,3,1,0.761062,0.075221,0.703814
10929,Agropecuario,veterinarias,6,2,0.889381,0.225664,0.688680
10363,Agropecuario,agronomo,50,3,1.000000,0.371681,0.628319
7085,Agropecuario,agropecuario,29,3,0.973451,0.371681,0.611638
2746,Agropecuario,agropecuaria,3,2,0.761062,0.225664,0.589318
11480,Agropecuario,agricultor,3,2,0.761062,0.225664,0.589318
5366,Agropecuario,sanidad,2,1,0.623894,0.075221,0.576964


In [49]:
agro.tail(10)

,Categoria,Palabras clave,Frecuencia al interior del grupo,Frecuencia entre los grupos,inter_rank,exter_rank,total_rank
11129,Agropecuario,analista,13,20,0.946903,0.964602,0.033519
6592,Agropecuario,auxiliar,11,20,0.929204,0.964602,0.032892
703,Agropecuario,asistente,1,19,0.278761,0.911504,0.024669
4317,Agropecuario,supervisor,1,19,0.278761,0.911504,0.024669
6310,Agropecuario,la,1,19,0.278761,0.911504,0.024669
9240,Agropecuario,bogota,1,19,0.278761,0.911504,0.024669
4878,Agropecuario,afines,1,20,0.278761,0.964602,0.009868
8996,Agropecuario,sector,1,20,0.278761,0.964602,0.009868
10564,Agropecuario,coordinador,2,21,0.623894,0.991150,0.005521
565,Agropecuario,profesional,7,22,0.915929,1.000000,0.000000


In [95]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

pd.set_option('display.float_format', '{:.10f}'.format)
scaler = MinMaxScaler()

resulting_df = pd.DataFrame(columns=filtered_data.columns)
for category in filtered_data['Categoria'].unique():
    df_new = filtered_data[filtered_data['Categoria']==category]
    df_new['inter_rank'] = scaler.fit_transform(np.array(df_new['Frecuencia al interior del grupo']).reshape(-1,1))
    df_new['exter_rank'] = scaler.fit_transform(np.array(df_new['Frecuencia entre los grupos']).reshape(-1,1))
    resulting_df = pd.concat([resulting_df, df_new])

resulting_df['total_rank'] = resulting_df.apply(lambda row: \
                                                row.inter_rank * (1-row.exter_rank), axis=1)
resulting_df

,Categoria,Palabras clave,Frecuencia al interior del grupo,Frecuencia entre los grupos,inter_rank,exter_rank,total_rank
0,Operativo,prodiccion,2,1,0.0001243781,0.0000000000,0.0001243781
1,Operativo,parrilleros,1,1,0.0000000000,0.0000000000,0.0000000000
2,Operativo,ingmecanico,1,1,0.0000000000,0.0000000000,0.0000000000
3,Operativo,documentacion,6,1,0.0006218905,0.0000000000,0.0006218905
7,Operativo,llamados,1,1,0.0000000000,0.0000000000,0.0000000000
...,...,...,...,...,...,...,...
11287,Juridicos,gestor,1,17,0.0000000000,0.7619047619,0.0000000000
11425,Juridicos,ingenieros,1,7,0.0000000000,0.2857142857,0.0000000000
11662,Juridicos,financiero,4,10,0.0241935484,0.4285714286,0.0138248848
11785,Juridicos,administrativo,2,16,0.0080645161,0.7142857143,0.0023041475


In [96]:
agro = resulting_df[resulting_df.Categoria == 'Agropecuario']
agro = agro.sort_values(["inter_rank"], ascending=False)

In [97]:
agro.head(10)

,Categoria,Palabras clave,Frecuencia al interior del grupo,Frecuencia entre los grupos,inter_rank,exter_rank,total_rank
10363,Agropecuario,agronomo,50,3,1.0000000000,0.0952380952,0.9047619048
2166,Agropecuario,ingeniero,44,19,0.8775510204,0.8571428571,0.1253644315
9410,Agropecuario,veterinario,30,4,0.5918367347,0.1428571429,0.5072886297
7085,Agropecuario,agropecuario,29,3,0.5714285714,0.0952380952,0.5170068027
1340,Agropecuario,tecnico,24,20,0.4693877551,0.9047619048,0.0447035957
1439,Agropecuario,asesor,13,18,0.2448979592,0.8095238095,0.0466472303
2902,Agropecuario,credito,13,9,0.2448979592,0.3809523810,0.1516034985
11129,Agropecuario,analista,13,20,0.2448979592,0.9047619048,0.0233236152
6592,Agropecuario,auxiliar,11,20,0.2040816327,0.9047619048,0.0194363460
565,Agropecuario,profesional,7,22,0.1224489796,1.0000000000,0.0000000000


In [98]:
agro.tail(10)

,Categoria,Palabras clave,Frecuencia al interior del grupo,Frecuencia entre los grupos,inter_rank,exter_rank,total_rank
8077,Agropecuario,clinica,1,5,0.0000000000,0.1904761905,0.0000000000
8260,Agropecuario,nutricional,1,2,0.0000000000,0.0476190476,0.0000000000
4190,Agropecuario,consulta,1,4,0.0000000000,0.1428571429,0.0000000000
4174,Agropecuario,trabajo,1,15,0.0000000000,0.6666666667,0.0000000000
8738,Agropecuario,estudios,1,9,0.0000000000,0.3809523810,0.0000000000
8947,Agropecuario,medicina,1,5,0.0000000000,0.1904761905,0.0000000000
8960,Agropecuario,key,1,2,0.0000000000,0.0476190476,0.0000000000
8996,Agropecuario,sector,1,20,0.0000000000,0.9047619048,0.0000000000
3630,Agropecuario,monteria,1,4,0.0000000000,0.1428571429,0.0000000000
9240,Agropecuario,bogota,1,19,0.0000000000,0.8571428571,0.0000000000


In [99]:
df_1 = filtered_data.copy()

In [109]:
pd.set_option('display.float_format', '{:.10f}'.format)

resulting_df = pd.DataFrame(columns=filtered_data.columns)
for category in filtered_data['Categoria'].unique():
    df_new = filtered_data[filtered_data['Categoria']==category]
    sm_new = df_new['Frecuencia al interior del grupo'].sum()
    df_new['pivot_rank'] = df_new['Frecuencia al interior del grupo'].apply(lambda x: x / sm_new)
    resulting_df = pd.concat([resulting_df, df_new])

resulting_df

,Categoria,Palabras clave,Frecuencia al interior del grupo,Frecuencia entre los grupos,pivot_rank
0,Operativo,prodiccion,2,1,0.0000294755
1,Operativo,parrilleros,1,1,0.0000147377
2,Operativo,ingmecanico,1,1,0.0000147377
3,Operativo,documentacion,6,1,0.0000884265
7,Operativo,llamados,1,1,0.0000147377
...,...,...,...,...,...
11287,Juridicos,gestor,1,17,0.0030211480
11425,Juridicos,ingenieros,1,7,0.0030211480
11662,Juridicos,financiero,4,10,0.0120845921
11785,Juridicos,administrativo,2,16,0.0060422961


In [111]:
agro = resulting_df[resulting_df.Categoria == 'Agropecuario']
agro = agro.sort_values(["pivot_rank"], ascending=False)

In [112]:
agro.head(10)

,Categoria,Palabras clave,Frecuencia al interior del grupo,Frecuencia entre los grupos,pivot_rank
10363,Agropecuario,agronomo,50,3,0.1168224299
2166,Agropecuario,ingeniero,44,19,0.1028037383
9410,Agropecuario,veterinario,30,4,0.0700934579
7085,Agropecuario,agropecuario,29,3,0.0677570093
1340,Agropecuario,tecnico,24,20,0.0560747664
1439,Agropecuario,asesor,13,18,0.0303738318
2902,Agropecuario,credito,13,9,0.0303738318
11129,Agropecuario,analista,13,20,0.0303738318
6592,Agropecuario,auxiliar,11,20,0.0257009346
565,Agropecuario,profesional,7,22,0.0163551402


In [113]:
agro.tail(10)

,Categoria,Palabras clave,Frecuencia al interior del grupo,Frecuencia entre los grupos,pivot_rank
8077,Agropecuario,clinica,1,5,0.0023364486
8260,Agropecuario,nutricional,1,2,0.0023364486
4190,Agropecuario,consulta,1,4,0.0023364486
4174,Agropecuario,trabajo,1,15,0.0023364486
8738,Agropecuario,estudios,1,9,0.0023364486
8947,Agropecuario,medicina,1,5,0.0023364486
8960,Agropecuario,key,1,2,0.0023364486
8996,Agropecuario,sector,1,20,0.0023364486
3630,Agropecuario,monteria,1,4,0.0023364486
9240,Agropecuario,bogota,1,19,0.0023364486


In [116]:
resulting_df.groupby('Palabras clave')['pivot_rank'].sum()

Palabras clave
abarrotes         0.0019091962
abastecedor       0.0000147377
abastecimiento    0.0113650353
abastecimientos   0.0009861933
abastesimiento    0.0009861933
                      ...     
zp                0.0002141548
zulia             0.0000185463
zunchado          0.0000294755
àrea              0.0001060108
òptica            0.0000147377
Name: pivot_rank, Length: 5963, dtype: float64